- Buat system rekomendasi content based dan collaborative filtering
- Tentukan daftar rekomendasi anime untuk:
    - Content based berdasarkan ```Genre```:
        - user 'Ali' baca anime no 1
        - user 'Bebi' baca anime no 2,24,25
        - user 'Cindy' baca anime no 10,200
        - user dengan ```user id = 1 ```
    - Collaborative:
        - user 'Ali' memberikan rating no 1:10
        - user 'Bebi' memberikan rating no 2: 8, no 24:-1, no 25:5
        - user 'Cindy' memberikan rating no 10:10, no 200:1
        - user dengan ```user id = 1 ```

# Content Based

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('anime.csv')
# df = df.iloc[:1000]
df2 = pd.read_csv('rating.csv')
df2.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [9]:
df.head(15)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [3]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [4]:
df = df.dropna()

df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
anime_id    12017 non-null int64
name        12017 non-null object
genre       12017 non-null object
type        12017 non-null object
episodes    12017 non-null object
rating      12017 non-null float64
members     12017 non-null int64
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


In [5]:
cv = CountVectorizer(tokenizer= lambda x: x.split(', '))
ca = cv.fit_transform(df['genre'])
ca = ca.toarray()
print(cv.get_feature_names())


# reduce float precision
# ca = ca.astype(np.int32)
ca

['action', 'adventure', 'cars', 'comedy', 'dementia', 'demons', 'drama', 'ecchi', 'fantasy', 'game', 'harem', 'hentai', 'historical', 'horror', 'josei', 'kids', 'magic', 'martial arts', 'mecha', 'military', 'music', 'mystery', 'parody', 'police', 'psychological', 'romance', 'samurai', 'school', 'sci-fi', 'seinen', 'shoujo', 'shoujo ai', 'shounen', 'shounen ai', 'slice of life', 'space', 'sports', 'super power', 'supernatural', 'thriller', 'vampire', 'yaoi', 'yuri']


array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [6]:
# 3. Cosine similarity dari tiap data di matrix freq

cosScore = cosine_similarity(ca)
cosScore

array([[1.        , 0.18898224, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.18898224, 1.        , 0.28571429, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.28571429, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ]])

## Joblib

In [10]:
import joblib

In [11]:
joblib.dump(cosScore, 'Rekomendasi')

['Rekomendasi']

### Rekomendasi Ali

In [13]:
Ali = df[df['anime_id'] == 1].index[0]

similarAli= list(enumerate(cosScore[Ali]))
# similarAli # (index, %similarity)

In [14]:
similarAli = sorted(similarAli, key=lambda x: x[1], reverse=True)
# similarAli

In [15]:
indexAli = []
for i in similarAli[:11]:
    x = i[0]
    indexAli.append(x)
indexAli

[22, 1465, 6565, 551, 1073, 1398, 2333, 2735, 3056, 5719, 700]

In [16]:
judulAli = []
for i in indexAli:
    x = df['name'].iloc[i]
    judulAli.append(x)
judulAli

['Cowboy Bebop',
 'Cowboy Bebop: Yose Atsume Blues',
 'Seihou Tenshi Angel Links',
 'Seihou Bukyou Outlaw Star',
 'Waga Seishun no Arcadia',
 'Waga Seishun no Arcadia: Mugen Kidou SSX',
 'Ginga Tetsudou Monogatari',
 'Uchuu Kaizoku Captain Harlock: Arcadia-gou no Nazo',
 'Ginga Tetsudou Monogatari: Eien e no Bunkiten',
 'Kaitei Choutokkyuu: Marine Express',
 'Urusei Yatsura Movie 2: Beautiful Dreamer']

In [17]:
dfAli = []
for i in similarAli[:11]:
    dfAli.append(df.iloc[i[0]])
dfAli = pd.DataFrame(dfAli)
dfAli

,anime_id,name,genre,type,episodes,rating,members
22,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1465,4037,Cowboy Bebop: Yose Atsume Blues,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Special,1,7.53,19565
6568,1226,Seihou Tenshi Angel Links,"Action, Adventure, Comedy, Drama, Romance, Sci...",TV,13,6.06,4817
551,400,Seihou Bukyou Outlaw Star,"Action, Adventure, Comedy, Sci-Fi, Space",TV,24,7.98,78600
1073,2202,Waga Seishun no Arcadia,"Action, Adventure, Drama, Sci-Fi, Space",Movie,1,7.70,4415
1398,2203,Waga Seishun no Arcadia: Mugen Kidou SSX,"Action, Adventure, Drama, Sci-Fi, Space",TV,22,7.56,2587
2333,1490,Ginga Tetsudou Monogatari,"Action, Adventure, Drama, Sci-Fi, Space",TV,26,7.29,5947
2735,2470,Uchuu Kaizoku Captain Harlock: Arcadia-gou no ...,"Action, Adventure, Drama, Sci-Fi, Space",Movie,1,7.18,2579
3057,2717,Ginga Tetsudou Monogatari: Eien e no Bunkiten,"Action, Adventure, Drama, Sci-Fi, Space",TV,24,7.09,2072
5721,3175,Kaitei Choutokkyuu: Marine Express,"Action, Adventure, Comedy, Drama, Sci-Fi",Special,1,6.37,551


### Rekomendasi Bebi

In [50]:
Bebi = df[df['anime_id'] == 25].index
# print(Bebi) # 0,464,1432

similarBebi1= list(enumerate(cosScore[0]))
similarBebi2= list(enumerate(cosScore[464]))
similarBebi3= list(enumerate(cosScore[1432]))

In [51]:
similarBebi1 = sorted(similarBebi1, key=lambda x: x[1], reverse=True)
similarBebi2 = sorted(similarBebi2, key=lambda x: x[1], reverse=True)
similarBebi3 = sorted(similarBebi3, key=lambda x: x[1], reverse=True)


In [52]:
dfBebi = []
for i in similarBebi1[:4],similarBebi2[:4],similarBebi3[:4]:
    dfBebi.append(df.iloc[i[0][0]])
    dfBebi.append(df.iloc[i[1][0]])
    dfBebi.append(df.iloc[i[2][0]])
dfBebi = pd.DataFrame(dfBebi)
dfBebi

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
5805,547,Wind: A Breath of Heart OVA,"Drama, Romance, School, Supernatural",OVA,3,6.35,2043
6394,546,Wind: A Breath of Heart (TV),"Drama, Romance, School, Supernatural",TV,13,6.14,7778
367,846,School Rumble Ni Gakki,"Comedy, Romance, School, Shounen",TV,26,8.15,93095
464,24,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553
772,517,School Rumble Ichi Gakki Hoshuu,"Comedy, Romance, School, Shounen",OVA,2,7.85,40868
1432,25,Sunabouzu,"Action, Adventure, Comedy, Ecchi, Sci-Fi, Shounen",TV,24,7.55,65516
4486,370,Burn Up Excess,"Action, Adventure, Comedy, Ecchi, Police, Sci-...",TV,13,6.71,6402
2760,33713,Seihou Bukyou Outlaw Star Specials,"Action, Adventure, Comedy, Ecchi, Sci-Fi",Special,2,7.16,1509


### Rekomendasi Cindy

In [87]:
## anime id = 100 dan 200
Cindy1 = df[df['anime_id']==100].index[0] #2210
Cindy2 = df[df['anime_id']==200].index[0] #1302

similarCindy1= list(enumerate(cosScore[Cindy1]))
similarCindy2= list(enumerate(cosScore[Cindy2]))
similarCindy1 = sorted(similarCindy1, key = lambda i: i[1], reverse=True)
similarCindy2 = sorted(similarCindy2, key = lambda i: i[1], reverse=True)
# similarCindy1

In [88]:
# index1 = []
# index2 = []
# for i in similarCindy1[:5]:
#     index1.append(i[0])
# for i in similarCindy2[:5]:
#     index2.append(i[0])
# index1,index2
    

([2210, 233, 534, 925, 1399], [1302, 4160, 4752, 184, 4403])

In [90]:
Cindy = similarCindy1+similarCindy2
Cindy = sorted(Cindy, key = lambda i: i[1], reverse=True)
# Cindy

In [94]:
index = []
for i in Cindy[:10]:
    index.append(i[0])
Cindy = df.iloc[index]
Cindy

,anime_id,name,genre,type,episodes,rating,members
1302,200,Tenshi na Konamaiki,"Comedy, Magic, Romance, Shounen",TV,50,7.60,15077
2210,100,Shin Shirayuki-hime Densetsu Prétear,"Comedy, Drama, Fantasy, Magic, Romance, Shoujo...",TV,13,7.33,37394
233,372,Cardcaptor Sakura Movie 2: Fuuin Sareta Card,"Comedy, Drama, Fantasy, Magic, Romance, Shoujo",Movie,1,8.28,60637
4162,626,Mamotte Shugogetten!,"Comedy, Fantasy, Magic, Romance, Shounen",TV,22,6.79,4995
4754,660,Denshin Mamotte Shugogetten,"Comedy, Fantasy, Magic, Romance, Shounen",OVA,8,6.64,2372
184,7674,Bakuman.,"Comedy, Romance, Shounen",TV,25,8.35,246899
4405,590,Kage kara Mamoru!,"Comedy, Romance, Shounen",TV,12,6.73,17136
7034,2081,Makeruna! Makendou,"Comedy, Magic, Shounen",OVA,1,5.78,555
534,996,Bishoujo Senshi Sailor Moon: Sailor Stars,"Adventure, Comedy, Drama, Fantasy, Magic, Roma...",TV,34,7.99,52586
925,997,Bishoujo Senshi Sailor Moon S: Kaguya Hime no ...,"Adventure, Comedy, Drama, Fantasy, Magic, Roma...",Movie,1,7.76,29394


### Rekomendasi User ID = 1

In [120]:
df2.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [127]:
Dedi = df2[df2['user_id']==1].sort_values(by='rating',ascending=False)
Dedi

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
101,1,15451,10
83,1,11757,10
106,1,16706,-1
...,...,...,...
52,1,8841,-1
53,1,9041,-1
54,1,9062,-1
55,1,9136,-1


User ID=1 memberikan rating 10 untuk 4 anime_id 8074,11617,15451, dan 11757, selain itu usir id 1 memberi rating -1 atau artinya dia menonton tapi tidak memberikan rating.

In [128]:
Dedi1 = Dedi['anime_id'].iloc[:4].index
Dedi1[0]

47

In [132]:
similarDedi1= list(enumerate(cosScore[Dedi1[0]]))
similarDedi2= list(enumerate(cosScore[Dedi1[1]]))
similarDedi3= list(enumerate(cosScore[Dedi1[2]]))
similarDedi4= list(enumerate(cosScore[Dedi1[3]]))

In [133]:
similarDedi = similarDedi1 + similarDedi2 + similarDedi3 + similarDedi4
similarDedi = sorted(similarDedi, key = lambda i: i[1], reverse=True)
similarDedi[:10]

[(47, 1.0000000000000002),
 (149, 1.0000000000000002),
 (83, 1.0000000000000002),
 (3029, 1.0000000000000002),
 (101, 0.9999999999999999),
 (268, 0.9999999999999999),
 (290, 0.9999999999999999),
 (461, 0.9999999999999999),
 (795, 0.9999999999999999),
 (3831, 0.9999999999999999)]

In [136]:
index = []
for i in similarDedi[:20]:
    index.append(i[0])
dfDedi = df.iloc[index]
dfDedi

,anime_id,name,genre,type,episodes,rating,members
47,22135,Ping Pong The Animation,"Psychological, Seinen, Sports",TV,11,8.67,97187
149,5040,One Outs,"Psychological, Seinen, Sports",TV,25,8.41,83056
83,877,Nana,"Comedy, Drama, Music, Romance, Shoujo, Slice o...",TV,47,8.55,197256
3030,1869,Nana Recaps,"Comedy, Drama, Music, Romance, Shoujo, Slice o...",Special,3,7.10,11447
101,18115,Magi: The Kingdom of Magic,"Action, Adventure, Fantasy, Magic, Shounen",TV,25,8.50,245026
268,14513,Magi: The Labyrinth of Magic,"Action, Adventure, Fantasy, Magic, Shounen",TV,25,8.24,317513
290,31741,Magi: Sinbad no Bouken (TV),"Action, Adventure, Fantasy, Magic, Shounen",TV,13,8.22,73480
461,22097,Magi: Sinbad no Bouken,"Action, Adventure, Fantasy, Magic, Shounen",OVA,5,8.06,52351
795,8086,Densetsu no Yuusha no Densetsu,"Action, Adventure, Fantasy, Magic, Shounen",TV,24,7.83,130689
3833,21235,Log Horizon Recap,"Action, Adventure, Fantasy, Magic, Shounen",Special,1,6.87,10245
